In [1]:
from __future__ import absolute_import
import argparse
import addm_toolbox 
import numpy as np
import pandas as pd
import os

import pkg_resources


In [2]:
subjectIds = [1]
subjectIds = ([str(subj) for subj in subjectIds] if subjectIds
                  else list(data))
trialsPerSubject = 120


In [3]:

expdata_file_name = '/Users/pradyumna/Documents/GiTs/FF2018/InputData_aDDM/aDDM_expdata_FF2018_Like_zScore_31.csv'
fixations_file_name = '/Users/pradyumna/Documents/GiTs/FF2018/InputData_aDDM/aDDM_Fix_FF2018_Like_YesBin_31.csv'
trials_file_name = '/Users/pradyumna/Documents/GiTs/FF2018/InputData_aDDM/trial_conditions_parcode_zScore.csv'

data = addm_toolbox.util.load_data_from_csv(expdata_file_name, fixations_file_name,
                              convertItemValues=None) # Tavares was transforming the values by default


In [4]:
dataTrials = list()
subjectIds = ([str(subj) for subj in subjectIds] if subjectIds
              else list(data))
for subjectId in subjectIds:
    maxNumTrials = len(data[subjectId]) // 2
    numTrials = (trialsPerSubject
                 if 1 <= trialsPerSubject <= maxNumTrials
                 else maxNumTrials)
    trialSet = np.random.choice(
        [trialId for trialId in range(len(data[subjectId]))
         if trialId % 2], numTrials, replace=False)
    dataTrials.extend([data[subjectId][t] for t in trialSet])

In [5]:
len(dataTrials)

60

# calculate likelihoods for the model

In [6]:
rangeD = [0.001, 0.005,0.01]
rangeSigma = [0.01,0.05 ,0.1]
rangeTheta = [0.1, 0.5, 0.9]

In [7]:
models = list()
for d in rangeD:
    for sigma in rangeSigma:
        for theta in rangeTheta:
            models.append(addm_toolbox.addm_cont.aDDM(d, sigma, theta))

In [8]:
likelihoods = dict()

for model in models:
    likelihoods[model.params] = model.parallel_get_likelihoods(
        dataTrials, numThreads=9)

In [9]:
NLL = dict()
for model in models:
      NLL[model.params] = - np.sum(np.log(likelihoods[model.params]))
optimalParams = min(NLL, key=NLL.get)

In [10]:
optimalParams

(0.001, 0.05, 0.9, 1)

# load conditions 

In [11]:
trialsFileName = pkg_resources.resource_filename(
            u"addm_toolbox", u"data/trial_conditions.csv")
trialConditions = addm_toolbox.util_cont.load_trial_conditions_from_csv(trials_file_name)

In [23]:
trialConditions

[(1.237880962, -1.136137321),
 (-1.81442826, -0.457846383),
 (-0.457846383, 0.559590024),
 (-0.118700914, -1.81442826),
 (-1.136137321, -1.136137321),
 (1.9161719, 0.559590024),
 (-0.118700914, 0.898735493),
 (1.237880962, 1.9161719),
 (-0.118700914, -0.457846383),
 (-1.136137321, 0.898735493),
 (0.898735493, 0.898735493),
 (0.220444555, -1.136137321),
 (-0.796991852, 0.898735493),
 (-0.457846383, -1.136137321),
 (0.559590024, -0.118700914),
 (-1.136137321, -0.457846383),
 (-0.796991852, -0.457846383),
 (-0.118700914, 1.577026431),
 (-0.457846383, 0.220444555),
 (-0.457846383, 0.898735493),
 (0.898735493, -0.118700914),
 (1.237880962, 1.577026431),
 (-0.457846383, -0.118700914),
 (-0.457846383, 0.220444555),
 (0.559590024, -0.457846383),
 (-1.136137321, -1.81442826),
 (-0.796991852, 1.9161719),
 (-0.796991852, 0.898735493),
 (-0.796991852, -1.136137321),
 (0.898735493, -1.81442826),
 (0.898735493, -1.136137321),
 (0.559590024, 0.220444555),
 (0.559590024, -0.796991852),
 (0.898735493, 

In [12]:
data['1'][119].valueRight

-1.2378809621020497

In [13]:
data['1'][119].valueLeft

-1.5770264311711084

## get distributions

In [14]:
fixationData =addm_toolbox.util_cont.get_empirical_distributions(
            data, subjectIds=subjectIds, useOddTrials=False, useEvenTrials=True,
            valueDiffs = list(range(-5,6,1)),fixDistType=u"linear")

In [15]:
fixationData.probFixLeftFirst

0.847457627118644

In [16]:
trialConditions

[(1.237880962, -1.136137321),
 (-1.81442826, -0.457846383),
 (-0.457846383, 0.559590024),
 (-0.118700914, -1.81442826),
 (-1.136137321, -1.136137321),
 (1.9161719, 0.559590024),
 (-0.118700914, 0.898735493),
 (1.237880962, 1.9161719),
 (-0.118700914, -0.457846383),
 (-1.136137321, 0.898735493),
 (0.898735493, 0.898735493),
 (0.220444555, -1.136137321),
 (-0.796991852, 0.898735493),
 (-0.457846383, -1.136137321),
 (0.559590024, -0.118700914),
 (-1.136137321, -0.457846383),
 (-0.796991852, -0.457846383),
 (-0.118700914, 1.577026431),
 (-0.457846383, 0.220444555),
 (-0.457846383, 0.898735493),
 (0.898735493, -0.118700914),
 (1.237880962, 1.577026431),
 (-0.457846383, -0.118700914),
 (-0.457846383, 0.220444555),
 (0.559590024, -0.457846383),
 (-1.136137321, -1.81442826),
 (-0.796991852, 1.9161719),
 (-0.796991852, 0.898735493),
 (-0.796991852, -1.136137321),
 (0.898735493, -1.81442826),
 (0.898735493, -1.136137321),
 (0.559590024, 0.220444555),
 (0.559590024, -0.796991852),
 (0.898735493, 

In [17]:
valueRight = trialConditions[0][0]
valueLeft = trialConditions[0][1]

In [18]:
fixationData.latencies

array([313, 290, 258, 230, 232, 247, 212, 215, 268, 225, 250, 299, 261,
       238, 234, 261, 237, 229, 241, 254, 983, 209, 212, 256,   0, 229,
       243, 214, 204, 233, 232, 244, 404, 244, 232, 252,   0, 250,   0,
         0, 227, 215, 202,   0, 206, 231, 241, 221, 208, 188, 232, 358,
         0, 240, 210,   0, 221, 208,   2])

In [19]:
simulTrials = list()

In [20]:
simulTrials.append( model.simulate_trial(valueLeft, valueRight, fixationData))
#print(a.valueLeft)
#print(a.valueRight)
#print(a.RT)
#print (a.choice)

In [21]:
simulTrials

## in this case values are still decimals

In [22]:
addm_toolbox.util_cont.save_simulations_to_csv(simulTrials,
                    "simul_expdata_TEST.csv",
                    "simul_fixations_TEST.csv")